In [0]:
%python
## Import libraries
import pyspark as ps
from pyspark.sql.functions import *
from pyspark.sql.types import *
from collections import defaultdict
from datetime import *
import pandas as pd
import json


In [0]:
class GetNgeMetadata:
  def __init__(self):
    self.query = "Query About to fill"
    self.sql_df = pd.DataFrame()
    self.metadata_json = {"Metadata":[]}    
  
  def logger(message):
    print(message)
    
  
  def set_query_string(self):
    query = "select distinct nsr.usecase_id,mu.Step_Function,mo.Sequence_Nr, mo.job_id, mo.Mapping_Params from com_us_alyt_ngebox.ib_ngebox_suggestion_requests nsr join com_us_alyt_ngebox.metadata_usecase mu on mu.usecase_id = nsr.usecase_id join com_us_alyt_ngebox.metadata_usecase_job mo on mo.usecase_id = mu.usecase_id order by mo.Sequence_Nr"
    self.query = query
  
  def run_query_and_set_sqldf(self):
    sql_df = sqlContext.sql(self.query).collect()
    self.sql_df = sql_df
    
  def format_sqldf_to_json(self):
    sql_output_rowlist = []
    for curr_row in self.sql_df: 
      sql_output_rowlist.append (curr_row.asDict(True))
    sql_df_new = pd.DataFrame(sql_output_rowlist)
    sql_df_new['Sequence_Nr']= sql_df_new['Sequence_Nr'].astype(str)
    #grouped_df = sql_df_new.groupby("job_id", group_keys=True).apply(lambda x: x)
    #grouped_df_ABC = grouped_df.groupby(['usecase_id','Step_Function'],as_index = False).agg({'job_id':','.join,'Mapping_Params':','.join,'Sequence_Nr':','.join})
    sql_jsonstr = sql_df_new.to_json(orient="records") # type(result) -> str
    sql_jsonparse = json.loads(sql_jsonstr) # type(parsed_json) -> list
    metadata_json = {"Metadata":sql_jsonparse} # type(metadata_json) -> dict
    data = metadata_json
    '''grouped_data = defaultdict(list)
    for item in data['Metadata']:
     grouped_data[item['']].append(item)
    result = {'Metadata': []}
    for key, values in grouped_data.items():
     group = {'Step_Function': key,'usecase_id': key, 'data': values}
     result['Metadata'].append(group)
    self.metadata_json = json.dumps(result)'''

    grouped_data = defaultdict(list)
    for item in data['Metadata']:
     key = (item['Step_Function'], item['usecase_id'])
     grouped_data[key].append(item)

    result = {'Metadata': []}
    for key, values in grouped_data.items():
     step_func, usecase_id = key
     group = {'Step_Function': step_func, 'usecase_id': usecase_id, 'data': values}
     result['Metadata'].append(group)
    self.metadata_json = json.dumps(result)

    
  def construct_microservice_response(self):
    resp = {
      "nge_response": {
        "status": 200,
        "body": self.metadata_json
      }
    }
    self.response = resp


In [0]:
metadata_obj = GetNgeMetadata()
metadata_obj.set_query_string()
metadata_obj.run_query_and_set_sqldf()
metadata_obj.format_sqldf_to_json()
metadata_obj.construct_microservice_response()
dbutils.notebook.exit(
  metadata_obj.response
)
#json.dumps(metadata_obj)
#from ppretty import ppretty
#print ppretty(A(), show_protected=True, show_static=True, show_properties=True)
#from pprint import pprint
#pprint(metadata_obj)

{'nge_response': {'status': 200, 'body': '{"Metadata": [{"Step_Function": "arn:aws:states:us-east-1:263789222982:stateMachine:NGE-Box-Usecase-Step-Function", "usecase_id": 511, "data": [{"usecase_id": 511, "Step_Function": "arn:aws:states:us-east-1:263789222982:stateMachine:NGE-Box-Usecase-Step-Function", "Sequence_Nr": "1", "job_id": "116908941694171", "Mapping_Params": "NA"}, {"usecase_id": 511, "Step_Function": "arn:aws:states:us-east-1:263789222982:stateMachine:NGE-Box-Usecase-Step-Function", "Sequence_Nr": "2", "job_id": "809892230969831", "Mapping_Params": "NA"}, {"usecase_id": 511, "Step_Function": "arn:aws:states:us-east-1:263789222982:stateMachine:NGE-Box-Usecase-Step-Function", "Sequence_Nr": "3", "job_id": "73435209655585", "Mapping_Params": "NA"}, {"usecase_id": 511, "Step_Function": "arn:aws:states:us-east-1:263789222982:stateMachine:NGE-Box-Usecase-Step-Function", "Sequence_Nr": "4", "job_id": "864957890369833", "Mapping_Params": "NA"}, {"usecase_id": 511, "Step_Function"